In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# from fancyimpute import mice
# from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler
from sklearn.preprocessing import Imputer
# from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import math
import sklearn.metrics as sklm
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

In [2]:
df_train_x = pd.read_csv('../data/train_dt.csv', index_col='row_id')
df_submission = pd.read_csv('../data/submission_format_klDJt5C.csv', index_col='row_id')

df_train_y = df_train_x.loc[:, 'prevalence_of_undernourishment']
df_train_x = df_train_x.drop(columns='prevalence_of_undernourishment')

print(df_train_y.shape)
print(df_train_x.shape)

(1401,)
(1401, 54)


In [3]:
df_test = pd.read_csv('../data/test_x_dt.csv', index_col='row_id')
df_test.shape

(616, 54)

# Feature selection by Lasso

In [12]:
from sklearn.linear_model import LassoCV

# lassocv = LassoCV(alphas=[0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100], max_iter=50000)
lassocv = LassoCV(alphas=[i/10000 for i in range(1000, 5000, 1)], max_iter=50000)

lassocv.fit(df_train_x, df_train_y)

mask = (lassocv.coef_ != 0)
df_train_x = df_train_x.loc[:, mask]
df_train_x.shape

c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


(1401, 36)

In [5]:
lassocv.alpha_

0.106

In [6]:
df_test = df_test.loc[:, mask]
df_test.shape

(616, 36)

# PCA
effect of PCA is not very well. As we decress dimansions, we would lose some information. 

# Standardization
Standardization is also not a good idea. it can also make our data lose some properties.

# Modeling

In [7]:
#Split train to 80% for training and 20% for validation
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df_train_x, df_train_y.values, test_size=0.2)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(1120, 36)
(1120,)
(281, 36)
(281,)


### Random Forest

In [8]:
# random forest
def rf_modeling(train_x, train_y):
    rf = RandomForestRegressor()
    param_grid = { 
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
    }

    grid_rf = GridSearchCV(rf, param_grid, n_jobs=-1, cv=5)
    forest_model = grid_rf.fit(train_x, train_y)
    
    return forest_model


forest_model = rf_modeling(train_x, train_y)
y_pred = forest_model.predict(test_x)


c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:1

c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:1

Root Mean Square Error = 2.811670697695787


c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:1

In [ ]:
print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(test_y, y_pred))))

In [9]:
forest_model = rf_modeling(df_train_x, df_train_y)

c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:1

c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:179: DeprecationWarning: 'min_samples_leaf' is deprecated in 0.20 and will be fixed to a value of 1 in 0.22.
  DeprecationWarning)
c:\users\student\appdata\local\programs\python\python36\lib\site-packages\sklearn\tree\tree.py:1

In [10]:
rf_result = forest_model.predict(df_test)
rf_result.shape

(616,)

In [11]:
df_submission['prevalence_of_undernourishment'] = pd.Series(rf_result)
df_submission.to_csv('rf_result.csv', index=True)

### XGBoost